In [1]:
from __future__ import print_function
import ipywidgets as widgets
from traitlets import Unicode, List, Float, Any, validate

In [2]:
class CanvasWidget(widgets.DOMWidget):
    _view_name = Unicode('CanvasView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    pixels = List(trait=Float()).tag(sync=True)

In [3]:
%%javascript
require.undef('hello');

define('canvas', ["@jupyter-widgets/base"], function(widgets) {
    
    var CanvasView = widgets.DOMWidgetView.extend({
        createCanvas: function() {
            var that = this;
            var width = 200,
                height = width;
            var $container = $('<div/>'),
                $canvas = $('<canvas/>')
                    .css("border", "1px solid black")
                    .attr("width", "" + width)
                    .attr("height", "" + height)
                    .mousedown(function(e){
                        paint = true;
                        var rect = this.getBoundingClientRect();
                        addClick(e.clientX - rect.left, e.clientY - rect.top);
                        redraw();
                    })
                    .mousemove(function(e){
                        if(paint){
                            var rect = this.getBoundingClientRect();
                            addClick(e.clientX - rect.left, e.clientY - rect.top, true);
                            redraw();
                        }
                    })
                    .mouseup(function(e){
                        paint = false;
                        that.model.set('pixels', getPixelData());
                        that.touch();
                    })
                    .mouseleave(function(e){
                        paint = false;
                        that.model.set('pixels', getPixelData());
                        that.touch();
                    }),
                $scaleCanvas = $('<canvas>')
                    .css("border", "1px solid black")
                    .attr("width", "" + width)
                    .attr("height", "" + height)[0],
                $clearButton = $('<button/>')
                    .text('clear canvas')
                    .click(function(e){
                        clearCanvas();
                    });
            
            var context = $canvas[0].getContext('2d'),
                scaleContext = $scaleCanvas.getContext("2d"),
                clickX = new Array(),
                clickY = new Array(),
                clickDrag = new Array(),
                paint = false;
            context.filter = 'blur(2px)';
            scaleContext.scale(0.1,0.1);

            function addClick(x, y, dragging){
                clickX.push(x);
                clickY.push(y);
                clickDrag.push(dragging);
            }

            function redraw(){
                context.clearRect(0, 0, context.canvas.width, context.canvas.height);
                context.rect(0, 0, context.canvas.width, context.canvas.height);
                context.fillStyle = 'white';
                context.fill();
                context.strokeStyle = "black";
                context.lineJoin = "round";
                context.lineWidth = 10;
                for(var i=0; i < clickX.length; i++) {
                    context.beginPath();
                    if(clickDrag[i] && i){
                        context.moveTo(clickX[i-1], clickY[i-1]);
                    } else {
                        context.moveTo(clickX[i]-1, clickY[i]);
                    }
                    context.lineTo(clickX[i], clickY[i]);
                    context.closePath();
                    context.stroke();
                }
            }

            function clearCanvas() {
                clickX = new Array();
                clickY = new Array();
                clickDrag = new Array();
                context.clearRect(0, 0, context.canvas.width, context.canvas.height);
                scaleContext.clearRect(0, 0, context.canvas.width, context.canvas.height);
            }

            function getPixelData() {
                scaleContext.drawImage($canvas[0], 0, 0);
                var data = scaleContext
                    .getImageData(0, 0,
                                  $canvas[0].width * 0.1,
                                  $canvas[0].height * 0.1)
                    .data;
                
                return Array.prototype.slice.call(data)
                    .filter(function(v,i,a) {
                        return i % 4 == 0;
                    })
                    .map(function(v,i,a) {
                        return v / 255;
                    });
            }

            return $container
                .append($canvas)
                .append($clearButton)
                .append($scaleCanvas);
        },
        render: function() {
            $(this.el).append(this.createCanvas());
        }
    });

    return {
        CanvasView : CanvasView
    };
});

<IPython.core.display.Javascript object>

In [4]:
w = CanvasWidget()
display(w)

CanvasWidget()

In [5]:
w.pixels

[]